In [ ]:
import pandas as pd
import numpy as np
import torch
import nltk
import re
import spacy
from torch import nn, optim
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, XLMRobertaForTokenClassification 
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from gensim.utils import simple_preprocess
from unidecode import unidecode
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer

In [ ]:
# comments = open('commentsBrazilNormalized.txt', 'r', encoding="utf8")
# dates = open('datesBrazil.txt', 'r', encoding="utf8")
# grades = open('gradesBrazil.txt', 'r', encoding="utf8")
# titles = open('titlesBrazil.txt', 'r', encoding="utf8")
comments = open('comments/comments_grand_hyatt_sao_paulo.txt', 'r', encoding="utf8")

In [ ]:
comments_data = comments.readlines()
# dates_data = dates.readlines()
# grades_data = grades.readlines()
# titles_data = titles.readlines()

In [ ]:
comments.close()
# dates.close()
# grades.close()
# titles.close()

In [ ]:
comments_list = [comment.strip() for comment in comments_data]
# dates_list = [date.strip() for date in dates_data]
# grades_list = [grade.strip() for grade in grades_data]
# titles_list = [title.strip() for title in titles_data]

In [ ]:
# filtered_comments = []
# hotel_indexes = []

# for i, comment in enumerate(comments_list):
#     comment_without_accents = unidecode(comment)
#     if re.search(r'\bcullinam\b', comment_without_accents, re.IGNORECASE):
#         filtered_comments.append(comment)
#         hotel_indexes.append(i)

# print(len(filtered_comments))

In [ ]:
# hotel_indexes

In [ ]:
# comments_list[569105]

In [ ]:
# selected_comments = []

# saint moritz: range(299, 1378) BRASÍLIA
# grand hyatt: range(462126, 462913) RIO, range(556258, 558881) SÃO PAULO
# royal tulip: range(11582, 13657) BRASÍLIA, range(387099, 387633) RIO
# copacabana palace: range(419735, 421697) RIO
# tivoli: range(566513, 567748) SÃO PAULO

# for index in range(566513, 567748):
#     selected_comments.append(comments_list[index])

In [ ]:
# len(selected_comments)

In [ ]:
# with open('comments_tivoli_sao_paulo.txt', 'w') as file:
#     for comment in selected_comments:
#         file.write(comment + '\n')

In [ ]:
# df = pd.DataFrame(list(zip(selected_comments, dates_list, grades_list, titles_list)), columns=['comments', 'dates', 'grades', 'titles'])
df = pd.DataFrame(list(zip(comments_list)))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
MAX_LEN = 175
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
nltk.download(["stopwords", "rslp"])

PT_STOPWORDS = nltk.corpus.stopwords.words('portuguese')

REGEX = [
    {'input': r'/\d+/g', 'output': r' '},               # removing digits
    {'input': r'[^\w\s]', 'output': r' '},              # removing punctuationon
    {'input': r'\n', 'output': r' '},                   # removing line breaks
    {'input': r'\b\w{1,2}\b', 'output': r''},           # removing short words (1 or 2 characters)
]

def regex_treatment(text):
    for pattern in REGEX:
        text = re.sub(pattern['input'], pattern['output'], text)
    return text.strip().lower()

def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in PT_STOPWORDS])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_md")

# Ignoring VERBs, ADVs, ADJs, PROPNs (...)
def lemmatizer(text, postags_permit=['NOUN']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permit])
    return doc_lemma

def remove_accentuation(text):
    return unidecode(text)

In [ ]:
df["comments_topic"] = df["comments"].apply(regex_treatment)
df["comments_topic"] = df["comments_topic"].apply(remove_stopwords)
df["comments_topic"] = df["comments_topic"].apply(lemmatizer)

In [ ]:
df["comments_topic"] = df["comments_topic"].apply(remove_accentuation)

In [ ]:
def remove_accentuation_lower(text):
    return unidecode(text.lower())

In [ ]:
df["comments_entity"] = df["comments"].apply(remove_accentuation_lower)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
wordcloud_text = ' '.join(df["comments_topic"].astype(str))

wordcloud = WordCloud(width=800, height=400, random_state=15, max_font_size=110, max_words=130, collocations=False).generate(wordcloud_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
df_comments = df["comments_topic"]
doc_comments = df_comments.to_list()
doc_comments[:5]

In [ ]:
df.to_csv("df_hotel.csv")
df_comments.to_csv("df_comments.csv")

In [ ]:
def extract_entities(df):
    all_entities = []

    for comment in df["comments_entity"]:
        predictions = ner_task(comment)
        comment_entities = []

        if predictions != []:
            combined_word = ""
            previous_entity = None

            for prediction in predictions:
                entity = prediction['entity']
                word = prediction['word']

                if not word.startswith('▁'):
                    combined_word += word
                else:
                    if combined_word:
                        comment_entities.append((combined_word, previous_entity))
                    combined_word = word

                previous_entity = entity

            if combined_word:
                comment_entities.append((combined_word, previous_entity))

        all_entities.append(comment_entities)

    df_copy = df.copy()
    df_copy["token_predictions"] = all_entities

    return df_copy

In [ ]:
df_ner = df.copy()
df_ner["token_predictions"] = ""

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, XLMRobertaForTokenClassification, pipeline, DistilBertTokenizerFast, BertForTokenClassification

MAX_LEN = 175
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

tokenizer_ner = AutoTokenizer.from_pretrained("51la5/roberta-large-NER", model_max_length=512)
model_ner = XLMRobertaForTokenClassification.from_pretrained("51la5/roberta-large-NER")
ner_task = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

In [ ]:
df_ner = extract_entities(df_ner)

In [ ]:
count_unique = df_ner['token_predictions'].explode().value_counts()
count_unique